In [1]:
#Imports
import pandas as pd
import numpy as np
import os
import glob
import subprocess

In [2]:
def y(df):

    return df[
        (df['DIA_NAC'] > 0) & (df['DIA_NAC'] <= 31) &
        (df['PESO'] > 0) & (df['PESO'] < 9999) & 
        (df['TALLA'] > 0) & (df['TALLA'] < 99) & 
        (df['EDAD_P'] > 0) & (df['EDAD_P'] < 99) &
        (df['EDAD_M'] > 0) & (df['EDAD_M'] < 99)
    ]

In [3]:
#1.a.I
def loadDataFramesFromCSV(folder, indexKey) :
        
    files = [os.path.basename(x) for x in glob.glob(folder)]
    
    mapData = []

    for fileName in files:
        mapData.append(pd.read_csv(folder[0: -5] + fileName, sep=';', index_col=None, header=0, error_bad_lines=False))
       
    return pd.concat(mapData)

In [4]:
#1.a.II
def mdb(folder):
    
    files = [os.path.basename(x) for x in glob.glob(folder)]
    
    mapData = []

    for fileName in files:
        tables = subprocess.check_output(["mdb-tables", os.getcwd() + "/datasets/nacimientos/" + fileName]).decode().split()
        
        for table in tables:
            d = subprocess.check_output(["mdb-export", os.getcwd() + "/datasets/nacimientos/" + fileName, table]).decode().split('\n')
            columns = d[0].split(',')
            data = [i.split(',') for i in d[1:]]
            mapData.append(pd.DataFrame(columns=columns, data=data))
            
    return pd.concat(mapData)

In [5]:
df1 = loadDataFramesFromCSV(os.getcwd() + "/datasets/nacimientos/*.csv", 4)
df2 = mdb(os.getcwd() + "/datasets/nacimientos/*.mdb").reset_index(drop = True)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
df2 = df2[
    (df2['PESO'].notnull()) &
    (df2['TALLA'].notnull())
]

df2['DIA_NAC'] = df2['DIA_NAC'].apply(lambda v: int(v))
df2['PESO'] = df2['PESO'].apply(lambda v: int(v))
df2['TALLA'] = df2['TALLA'].apply(lambda v: int(v))
df2['EDAD_P'] = df2['EDAD_P'].apply(lambda v: int(v))
df2['EDAD_M'] = df2['EDAD_M'].apply(lambda v: int(v))

In [7]:
dfs = []
dfs.append(y(df1))
dfs.append(y(df2))

nacimientos = pd.concat(dfs)

In [8]:
#1.b
nacimientos['MES_NAC'].mode()
#El mes de nacimiento mas comun, es el mes de Enero, dentro de todos los dataframes, asumiendo 
#los predicados propuestos en consulta general

0    1
dtype: object

In [9]:
#1.c
nacimientos['DIA_NAC'].mode()
#El dia de nacimiento mas comun, es el dia 5, asumiendo los predicados propuestos en consulta general

0    5
dtype: int64

In [10]:
covar_general = nacimientos.cov()
correl_general = nacimientos.corr()

In [11]:
#1.d.I
covar_general['PESO']['TALLA']

#La covarianza entre peso y talla de todo el dataframe es 1124.43, esto quiere decir que hay fuerte 
#relacion entre las dos variables

1124.4317461402002

In [12]:
#1.d.II
correl_general['PESO']['TALLA']

#Se observa que la correlacion entre las variables y entre todos los años de estudio, se mantiene 
#como asociacion alta, siendo esta de 0.8093

0.8093616617287183

In [13]:
def covar(var1, var2) :

    dataframe_groupped = nacimientos.groupby(by = ['ANO_NAC']).agg(
        d_max=pd.NamedAgg(column="ANO_NAC", aggfunc="max")
    )
     
    anios = dataframe_groupped.reset_index()['ANO_NAC']
    
    for anio in anios:
        
        covar_vars = nacimientos[
            nacimientos['ANO_NAC'] == anio
        ].cov()[var1][var2]
        
        print("año:" , anio , "covarianza", var1, var2, covar_vars)

In [14]:
def correl(var1, var2) :

    dataframe_groupped = nacimientos.groupby(by = ['ANO_NAC']).agg(
        d_max=pd.NamedAgg(column="ANO_NAC", aggfunc="max")
    )
     
    anios = dataframe_groupped.reset_index()['ANO_NAC']
    
    for anio in anios:
        
        correl_vars = nacimientos[
            nacimientos['ANO_NAC'] == anio
        ].corr()[var1][var2]
        
        print("año:" , anio , "correlacion", var1, var2, correl_vars)

In [15]:
#1.d.III
covar('PESO', 'TALLA')

#Se observa que la covarianza, entre todos los años, se mantiene fuerte entre las variables peso-talla, 
#es decir la relacion entre estas variables es fuerte y se mantiene uniforme

año: 1996 covarianza PESO TALLA 1085.4653360823058
año: 1997 covarianza PESO TALLA 1088.3400018104376
año: 1998 covarianza PESO TALLA 1124.1344344440379
año: 1999 covarianza PESO TALLA 1137.0183582782013
año: 2000 covarianza PESO TALLA 1138.4201712119925
año: 2001 covarianza PESO TALLA 1147.7090440657344
año: 2002 covarianza PESO TALLA 1169.0082515832255
año: 2003 covarianza PESO TALLA 1198.7999655229937
año: 2004 covarianza PESO TALLA 1188.5271018882
año: 2005 covarianza PESO TALLA 1160.3117396659222
año: 2006 covarianza PESO TALLA 1201.6511336237372
año: 2007 covarianza PESO TALLA 1175.6471654582335
año: 2008 covarianza PESO TALLA 1120.9805930010439
año: 2009 covarianza PESO TALLA 1073.41283596261
año: 2012 covarianza PESO TALLA 1113.94884718905
año: 2013 covarianza PESO TALLA 1122.3242014210202
año: 2015 covarianza PESO TALLA 1222.3854692961659
año: 2016 covarianza PESO TALLA 1206.5779894345908
año: 2017 covarianza PESO TALLA 1206.4964545510777
año: 1990 covarianza PESO TALLA 1033.9

In [16]:
#1.d.IV
correl('PESO', 'TALLA')

#Se observa que la correlacion entre las variables y entre los años de estudio, se mantiene como asociacion alta

año: 1996 correlacion PESO TALLA 0.8069518187611212
año: 1997 correlacion PESO TALLA 0.8089060125742329
año: 1998 correlacion PESO TALLA 0.8182298157917199
año: 1999 correlacion PESO TALLA 0.8177874503114567
año: 2000 correlacion PESO TALLA 0.8139722864561552
año: 2001 correlacion PESO TALLA 0.811966261784761
año: 2002 correlacion PESO TALLA 0.8091638292938252
año: 2003 correlacion PESO TALLA 0.8094935548666183
año: 2004 correlacion PESO TALLA 0.806082773838522
año: 2005 correlacion PESO TALLA 0.8048940634176609
año: 2006 correlacion PESO TALLA 0.8096761210853728
año: 2007 correlacion PESO TALLA 0.8067258011262431
año: 2008 correlacion PESO TALLA 0.8219082360418786
año: 2009 correlacion PESO TALLA 0.8240821791922216
año: 2012 correlacion PESO TALLA 0.8417518005847248
año: 2013 correlacion PESO TALLA 0.8404412586738272
año: 2015 correlacion PESO TALLA 0.816631793824781
año: 2016 correlacion PESO TALLA 0.7976631179418455
año: 2017 correlacion PESO TALLA 0.8307449477043788
año: 1990 corre

In [17]:
#1.e.I
covar_general['EDAD_P']['EDAD_M']

#La covarianza entre la edad del padre y edad de la madre de todo el dataframe es 35.3330, esto quiere 
#decir que hay fuerte relacion entre las dos variables de estudio

35.33309108434882

In [18]:
#1.e.II
correl_general['EDAD_P']['EDAD_M']

#Se observa que la correlacion entre las variables y entre todos los años de estudio, se mantiene 
#como asociacion alta, siendo esta de 0.7249

0.7249310826451041

In [19]:
#1.e.III
covar('EDAD_P', 'EDAD_M')

#Se observa que la covarianza, entre todos los años, se mantiene fuerte entre las variables 
#edad del padre y edad de la madre, es deci la relacion entre estas variables es fuerte

año: 1996 covarianza EDAD_P EDAD_M 33.15393468330035
año: 1997 covarianza EDAD_P EDAD_M 33.896385331567615
año: 1998 covarianza EDAD_P EDAD_M 34.678934435019116
año: 1999 covarianza EDAD_P EDAD_M 35.21948537746607
año: 2000 covarianza EDAD_P EDAD_M 36.028545323498555
año: 2001 covarianza EDAD_P EDAD_M 37.01806604993651
año: 2002 covarianza EDAD_P EDAD_M 37.3353295798695
año: 2003 covarianza EDAD_P EDAD_M 37.29463115107567
año: 2004 covarianza EDAD_P EDAD_M 36.766141555508945
año: 2005 covarianza EDAD_P EDAD_M 38.777586017939015
año: 2006 covarianza EDAD_P EDAD_M 39.14051584360459
año: 2007 covarianza EDAD_P EDAD_M 38.95771626706872
año: 2008 covarianza EDAD_P EDAD_M 39.36094388341994
año: 2009 covarianza EDAD_P EDAD_M 39.487466976679976
año: 2012 covarianza EDAD_P EDAD_M 39.52664728517216
año: 2013 covarianza EDAD_P EDAD_M 38.78129406704529
año: 2015 covarianza EDAD_P EDAD_M 36.61025838603627
año: 2016 covarianza EDAD_P EDAD_M 35.280384108889166
año: 2017 covarianza EDAD_P EDAD_M 33.92

In [20]:
#1.e.IV
correl('EDAD_P', 'EDAD_M')

#Se observa que la correlacion entre las variables de la edad del padre y de la madre, tienen alta relacion, 
#respecto a cada año de estudio.

año: 1996 correlacion EDAD_P EDAD_M 0.7191033622249087
año: 1997 correlacion EDAD_P EDAD_M 0.7247403683862226
año: 1998 correlacion EDAD_P EDAD_M 0.7296278053762393
año: 1999 correlacion EDAD_P EDAD_M 0.72940470423727
año: 2000 correlacion EDAD_P EDAD_M 0.7333586527520604
año: 2001 correlacion EDAD_P EDAD_M 0.737944562598619
año: 2002 correlacion EDAD_P EDAD_M 0.7385779153676287
año: 2003 correlacion EDAD_P EDAD_M 0.736209782351795
año: 2004 correlacion EDAD_P EDAD_M 0.7159289710161831
año: 2005 correlacion EDAD_P EDAD_M 0.7440481912650658
año: 2006 correlacion EDAD_P EDAD_M 0.7307681055553491
año: 2007 correlacion EDAD_P EDAD_M 0.7322004012098318
año: 2008 correlacion EDAD_P EDAD_M 0.7366510318168198
año: 2009 correlacion EDAD_P EDAD_M 0.7394125909248829
año: 2012 correlacion EDAD_P EDAD_M 0.7480015240640328
año: 2013 correlacion EDAD_P EDAD_M 0.7455483836770453
año: 2015 correlacion EDAD_P EDAD_M 0.7375315938224521
año: 2016 correlacion EDAD_P EDAD_M 0.7303860541985884
año: 2017 corr